In [2]:
import typer
import pandas as pd
import spacy
from pathlib import Path
from spacy.matcher import Matcher
from spacy.tokens import Span, Token
from spacy.util import filter_spans
from spacy.tokens import DocBin
from spacy import displacy
# from sklearn.model_selection import train_test_split
import re
import json

from adept.config import unit_registry
from adept.components.registry import ComponentsRegistry
from adept.utils.expand import ExpandSpan
import numpy as np
from adept.components.registry import ComponentsRegistry
from adept.components.sentencizer import Sentencizer
from adept.components.numeric import (NumericDimension, NumericExpand, NumericFraction, NumericMeasurement, NumericRange)
from adept.components.anatomical import AnatomicalEntity
from adept.components.traits import DiscreteTraitsEntity
from adept.components.traits import CustomTraitsEntity
from adept.preprocess import Preprocess
from adept.postprocess import Postproccess
from adept.traits import Traits
from adept.fields import Fields, Field
from adept.pipeline import Pipeline
from adept.config import LOG_DIR, logger, DATA_DIR, PROCESSED_DATA_DIR
from adept.utils.helpers import token_get_ent
from sklearn.model_selection import train_test_split

/Users/ben/Projects/NaturalHistoryMuseum/ADEPT/ADEPT/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
nlp = spacy.load("en_core_web_trf")
registry = ComponentsRegistry(nlp)
registry.add_components([
    Sentencizer,
    NumericExpand,
    NumericDimension,
    NumericMeasurement,
    NumericRange,
    NumericFraction,    
])

INIT custom_sentencizer
INIT numeric_expand
INIT numeric_dimensions
INIT numeric_measurements
^cm$|^ft$|^m$|^meter$|^metre$|^km$|^kilometer$|^kilometre$|^centimeter$|^centimetre$|^mm$|^millimeter$|^millimetre$|^um$|^micrometer$|^micrometre$|^micron$|^nm$|^nanometer$|^nanometre$|^pm$|^inch$
INIT numeric_range
INIT numeric_fraction


In [39]:
df = pd.read_csv(PROCESSED_DATA_DIR / 'af4b0c64764555e29e5f5a7847683c2f.descriptions.csv')



In [40]:
preprocess = Preprocess()

In [41]:
class EntityParser():
    # ('Who is Shaka Khan?', {
    #     'entities': [(7, 17, 'PERSON')]
    # }),  
    
    # 3-flowered etc.,     
    re_num_cadj = re.compile(r'[0-9]+\-[a-z]+')
    
    ent_labels = ['CARDINAL', 'QUANTITY']
    
    def __call__(self, doc):
        self.sents = []
        self.doc = doc

        for sent in doc.sents:
            
            
            
            self.ents = []
            self.error = False
            
            if self.sent_is_2n(sent):
                continue
                
            # try:
            #     if sent[3].text != 'basal':
            #         continue
            # except:
            #     continue
                
#             print(sent)
#             print(sent._.measurements)

            if sent._.dimensions:
                self.add_entities(sent, sent._.dimensions, 'DIMENSION')
            if sent._.measurements:
                self.add_entities(sent, sent._.measurements, 'MEASUREMENT') 
            if sent._.volume_measurements:
                self.add_entities(sent, sent._.volume_measurements, 'VOLUME')
            
            numeric_ents = [ent for ent in sent.ents if ent.label_ in ['CARDINAL', 'QUANTITY'] and not (ent[0]._.is_measurement or ent[0]._.is_dimension)]

            # Extract 3-flowered
            for m in self.re_num_cadj.finditer(sent.text): 
                span = sent.char_span(m.start(), m.end())

                # No span if the start/end doesn't align with token boundaries
                # Skip it - sent will fail with an existing numeric ent                 
                if not span:
                    continue
                    
                for token in span:
                    if ent := token_get_ent(token, self.ent_labels):
                        if ent in numeric_ents:
                            # Ensure the start chars are the same - otherwise we will have
                            # 3- *7-veined*                             
                            if ent.start_char == m.start():
                                numeric_ents.remove(ent)
                        
                self.add_entity(sent, span, 'NUMERIC_CADJ')
            
            if numeric_ents:
                for ent in numeric_ents[:]:  
                    # Match valid numbers e.g. 5-7 (-9)                     
                    if re.match(r'^[<>0-9\(\)\-\.\s]+$', ent.text):
                        # We're just going to use CARDINAL, rather than CARDINAL and QUANTITY
                        # as there's only two QUANTITY                         
                        self.add_entity(sent, ent, 'CARDINAL')
                        numeric_ents.remove(ent)  
            
            # If we removed all the numeric entities with our own, add it                 
            if not (numeric_ents or self.error) and self.ents:  
                sent_doc = sent.as_doc()
                # Remove any overlapping - e.g. if we have measurement that is also flagged as a dimension                 
                sent_doc.ents = filter_spans(self.ents)
                self.sents.append(sent_doc)
                
        return self.sents
                        
                    
    def add_entities(self, sent, spans, label):
        for span in spans:
            self.add_entity(sent, span, label)
    
    def add_entity(self, sent, span, label): 
        
        start = span.start - sent.start
        end = span.end - sent.start
        
        ent = Span(sent.as_doc(), start, end, label=label)  

        if not self.validate_parentheses(ent.text):
            print('INVALID PARENTHESES: ', ent.text)
            self.error = True
            return
        
        self.ents.append(ent)
        
    @staticmethod
    def validate_parentheses(text):
        return text.count('(') == text.count(')')
    
    @staticmethod
    def sent_is_2n(sent):
        return '2n' in [s.text for s in sent]

        
parser = EntityParser()   

# sample = df.sample(200)

# sample = df[df.taxon == 'Rotala ramosior']

train_data = []

for row in df.itertuples():
    print(row.taxon)    
    text = preprocess(row.description)   

    
    # Optional - to debug
    # print(row.taxon)     
    # displacy.render(sents, style="ent")
    
    doc = nlp(text)
    sents = parser(doc)
    
    for sent in sents:
        entities = [(ent.start_char, ent.end_char, ent.label_) for ent in sent.ents]     
        train_data.append((sent.text, {'entities': entities}))
        

# print(len(train_data))
        

            


Eleocharis palustris
INVALID PARENTHESES:  (1-4 mm
Eleocharis palustris


Error parsing range some 3-fid in styles 2-fid, very rarely some 3-fid.


INVALID PARENTHESES:  2001)
INVALID PARENTHESES:  1968)
Eleocharis palustris
Eleocharis palustris
Leersia hexandra
Phalaris arundinacea
Phalaris arundinacea
Phalaris arundinacea
Leersia oryzoides
Leersia oryzoides
Leersia oryzoides
Alopecurus aequalis
Alopecurus aequalis
Alopecurus aequalis
Ranunculus sceleratus
Ranunculus sceleratus
Ranunculus sceleratus
Ranunculus sceleratus
Potamogeton perfoliatus
Potamogeton perfoliatus
Potamogeton perfoliatus
Potamogeton perfoliatus
Eleocharis acicularis
Eleocharis acicularis
INVALID PARENTHESES:  1975)
INVALID PARENTHESES:  3)
INVALID PARENTHESES:  539)
Bolboschoenus maritimus
Bolboschoenus maritimus
Bolboschoenus maritimus
Myriophyllum verticillatum
Myriophyllum verticillatum
INVALID PARENTHESES:  1998)
Myriophyllum verticillatum
Myriophyllum verticillatum
Arundo donax
Arundo donax
Epilobium hirsutum
Epilobium hirsutum
Epilobium hirsutum
Epilobium hirsutum


Error parsing range 6-l 0 in Sepals 6-l 0 x 2-2.8 mm, apiculate, densely pubescent.
Error parsing range 3-l 0 cm in Capsules 3-l 0 cm long, on pedicels 0.5-2 cm long, villous.


Persicaria lapathifolia
Persicaria lapathifolia
Persicaria lapathifolia
Epilobium parviflorum
Epilobium parviflorum
Epilobium parviflorum
Epilobium parviflorum
Catabrosa aquatica
Catabrosa aquatica
Catabrosa aquatica
INVALID PARENTHESES:  (up to 5 cm
INVALID PARENTHESES:  (1.5-1.7 mm
Potentilla supina
Hippuris vulgaris
Hippuris vulgaris
Hippuris vulgaris
Nuphar pumila
Nuphar pumila
Potamogeton friesii
Potamogeton friesii
Potamogeton friesii
Veronica peregrina
Veronica peregrina
Ischaemum rugosum


...

Lemna minor
Lemna minor
Lemna minor
Pistia stratiotes
INVALID PARENTHESES:  4(3
INVALID PARENTHESES:  ) 1
INVALID PARENTHESES:  2)
Pistia stratiotes
Pistia stratiotes
Ceratophyllum demersum
Ceratophyllum demersum
INVALID PARENTHESES:  ) 3.5-6 x 2-4 x 1-2.5 mm
INVALID PARENTHESES:  ) 3.5-6 x 2-4 x 1-2.5 mm
Ceratophyllum demersum
INVALID PARENTHESES:  ) 3.5-6 x 2-4 mm
INVALID PARENTHESES:  ) 0.5-14 mm
Ceratophyllum demersum
Commelina benghalensis
Commelina benghalensis
Commelina benghalensis
Ipomoea aquatica
Ipomoea aquatica
Cyperus compressus
Cyperus compressus
Cyperus compressus
Cyperus rotundus


Error parsing range (as low as -18degC). in Cyperus esculentus is able to tolerate lower air temperatures (as low as -18degC).


INVALID PARENTHESES:  1987)
Cyperus rotundus
Cyperus rotundus
Cyperus squarrosus
Cyperus squarrosus
Cyperus squarrosus
INVALID PARENTHESES:  1974)
Cyperus polystachyos
Aeschynomene indica
Aeschynomene indica
Myriophyllum spicatum
Myriophyllum spicatum
INVALID PARENTHESES:  (3
INVALID PARENTHESES:  5)
INVALID PARENTHESES:  (1919
INVALID PARENTHESES:  1947)
Myriophyllum spicatum
INVALID PARENTHESES:  5)
Myriophyllum spicatum
Najas marina
Najas marina
Najas marina
Najas marina
Vallisneria spiralis
Spirodela polyrhiza
Spirodela polyrhiza
Potamogeton crispus
Potamogeton crispus
Potamogeton crispus
Potamogeton pusillus
Potamogeton pusillus
Potamogeton pusillus
Potamogeton pusillus
Stuckenia pectinata
Stuckenia pectinata
Trapa natans
Trapa natans
Trapa natans
Typha domingensis
Typha domingensis
Typha domingensis
Crotalaria lanceolata
Eclipta prostrata
Epilobium ciliatum
INVALID PARENTHESES:  (1.5-3 mm
Epilobium ciliatum
Epilobium ciliatum
Kyllinga brevifolia
Kyllinga brevifolia
Kyllinga brevi

Error parsing range two-thirds in lower glume lanceolate, 1.2-2 mm long, two-thirds to four-fifths as long as the spikelet (variable even in the same panicle), acute;


Sesuvium portulacastrum
Sesuvium portulacastrum
Canavalia rosea
Cardiospermum microcarpum
Triumfetta semitriloba
Triumfetta semitriloba
Eugenia uniflora
INVALID PARENTHESES:  (1
INVALID PARENTHESES:  ) 2-6-
Eugenia uniflora


In [42]:
# train, test = train_test_split(train_data,test_size=0.2)


# split_data = {
#     'train': train,
#     'test': test
# }

# for t, data in split_data.items():
    
#     print(f'Output {len(data)} {t} records')

    
#     print(file_path)

file_path = DATA_DIR / 'assets' / f'train_numeric.json'
with file_path.open('w') as f:
    f.write(json.dumps(train_data))    

In [8]:
len(train_data)

6843

In [79]:
sent, entities = sent_entities[0]

# ents = [Span(sent.as_doc(), start, end, label=string_id)]   

# [Span(sent.as_doc(), start, end, label=label) for start, end, label in entities['entities']]

# for t in sent:
#     print(t)
    
d = sent.as_doc()

# type(d.text[10])

print(entities['entities'])
print(sent)

# print(entities)

    
# Should we be using word for entities?????     

# displacy.render(sent, style="ent")

entities

for token in sent:
    print(dir(token))
    # print(token)
    # print(token.i)
    # print(token.idx)
    # print('-')
    
    # break



[(12, 20, 'MEASUREMENT')]
Pseudostems 20-40 cm.
['_', '__bytes__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', 'ancestors', 'check_flag', 'children', 'cluster', 'conjuncts', 'dep', 'dep_', 'doc', 'ent_id', 'ent_id_', 'ent_iob', 'ent_iob_', 'ent_kb_id', 'ent_kb_id_', 'ent_type', 'ent_type_', 'get_extension', 'has_dep', 'has_extension', 'has_head', 'has_morph', 'has_vector', 'head', 'i', 'idx', 'iob_strings', 'is_alpha', 'is_ancestor', 'is_ascii', 'is_bracket', 'is_currency', 'is_digit', 'is_left_punct', 'is_lower', 'is_oov', 'is_punct', 'is_quote', 'is_right_punct', 'is_sent_end', 'is_sent_start', 'is_space', 'is_stop', 'is_title', 'is_upper', 'lang', 'lang_', 'left_edge', 'lefts', 'lemma'

In [42]:
entities

[('Pseudostems 20-40 cm.', {'entities': [(12, 20, 'MEASUREMENT')]}),
 ('Leaves sessile except for 2 apical ones;',
  {'entities': [(26, 27, 'CARDINAL')]}),
 ('ligule brownish, 1-5 cm, sparsely appressed villous;',
  {'entities': [(17, 23, 'MEASUREMENT')]}),
 ('leaf blade obovate or oblong, 15-27 x 8-10 cm, adaxially glabrous, abaxially appressed yellow villous along midvein, base cuneate, apex acute.',
  {'entities': [(30, 45, 'DIMENSION')]}),
 ('Spikes 8-14 x 4-6 cm;', {'entities': [(7, 20, 'DIMENSION')]}),
 ('bracts rusty, ovate-lanceolate, 3-4.6 x 1.2-2 cm, membranous, appressed yellow villous, 3-flowered;',
  {'entities': [(32, 48, 'DIMENSION'), (88, 98, 'NUMERIC_CADJ')]}),
 ('bracteoles tubular, 2.7-3 cm, appressed yellow villous.',
  {'entities': [(20, 28, 'MEASUREMENT')]}),
 ('Calyx brownish, 3.3-3.5 cm, slightly split on 1 side, appressed yellow villous.',
  {'entities': [(16, 26, 'MEASUREMENT'), (46, 47, 'CARDINAL')]}),
 ('Corolla tube ca. 4 cm, glabrous;', {'entities': [(13, 

3-flowered
2-lobed
2-cleft


In [14]:


        

parser = EntityParser()
parser(doc)

print(parser.entities)
        
    

    # print(sent._.dimensions)

Rhizomes creeping.
[]
Pseudostems 20-40 cm.
[]
Leaves sessile except for 2 apical ones;
[]
ligule brownish, 1-5 cm, sparsely appressed villous;
[]
leaf blade obovate or oblong, 15-27 x 8-10 cm, adaxially glabrous, abaxially appressed yellow villous along midvein, base cuneate, apex acute.
[]
Spikes 8-14 x 4-6 cm;
[]
rachis appressed yellow villous;
[]
bracts rusty, ovate-lanceolate, 3-4.6 x 1.2-2 cm, membranous, appressed yellow villous, 3-flowered;
3-flowered
[]
bracteoles tubular, 2.7-3 cm, appressed yellow villous.
[]
Flowers white, slightly fragrant.
[]
Calyx brownish, 3.3-3.5 cm, slightly split on 1 side, appressed yellow villous.
[]
Corolla tube ca. 4 cm, glabrous;
[]
lobes linear, 2.5-3 cm x 3-4 mm.
[]
Lateral staminodes oblanceolate, ca. 2.3 cm x 7 mm.
[]
Labellum broadly ovate, ca. 2.2 x 1.9 cm, apically slightly 2-lobed or 2-cleft for ca. 0.5 its length.
2-lobed
2-cleft
[ca. 0.5 its length]
Filament white, equaling labellum;
[]
anther ca. 9 mm.
[]
Ovary ca. 5 mm, appressed ye

In [54]:
import re

In [18]:
df = pd.read_csv('/Users/ben/Projects/NaturalHistoryMuseum/Traits/traits/traits/corpus/preprocessed-descriptions.csv')

In [18]:
round(0.3,2)

0.3

In [17]:
re_num_cadj = re.compile(r'[0-9]+\-[a-z]+')

text = 'Capsules globose, 2-3 mm in diameter. , (3 or)4-valved, slightly exserted from floral tube.'

for m in re_num_cadj.finditer(text):
    print(m)
    

<re.Match object; span=(46, 54), match='4-valved'>


In [18]:
text[46:54]

'4-valved'

In [17]:
s = "{'entities': [[44, 50, 'MEASUREMENT']]}"

In [18]:
m = re.findall(r"[A-Z]{2,}", s)
label = m[0] if len(m) == 1 else 'MULTI'
label

'MEASUREMENT'

In [45]:
p = DATA_DIR / 'assets' / 'train_numeric.json'

df = pd.read_json(p)
df.columns = ['sent', 'entities']
re_upper_case = re.compile(r"[A-Z]{2,}")

def _get_label(entities):
    labels = {e[2] for e in entities['entities']}
    return next(iter(labels)) if len(labels) == 1 else 'MULTI'

df['label'] = df['entities'].apply(_get_label)

In [46]:
df['label'].value_counts()

MEASUREMENT     3686
CARDINAL        1164
MULTI            978
DIMENSION        831
NUMERIC_CADJ     184
Name: label, dtype: int64

In [52]:
train, test = train_test_split(df, test_size=0.3, random_state=0, stratify=df[['label']])

In [55]:
test, val = train_test_split(test, test_size=0.5, random_state=0, stratify=test[['label']])

In [58]:
df['label'].value_counts()

MEASUREMENT     3686
CARDINAL        1164
MULTI            978
DIMENSION        831
NUMERIC_CADJ     184
Name: label, dtype: int64

In [61]:
for sent, annot in df[['sent', 'entities']].values:
    print(annot)

{'entities': [[55, 68, 'MEASUREMENT']]}
{'entities': [[25, 30, 'MEASUREMENT']]}
{'entities': [[64, 76, 'MEASUREMENT']]}
{'entities': [[9, 12, 'CARDINAL'], [54, 55, 'CARDINAL']]}
{'entities': [[8, 9, 'CARDINAL']]}
{'entities': [[8, 18, 'MEASUREMENT']]}
{'entities': [[8, 9, 'CARDINAL']]}
{'entities': [[31, 42, 'MEASUREMENT']]}
{'entities': [[31, 41, 'MEASUREMENT']]}
{'entities': [[4, 19, 'MEASUREMENT']]}
{'entities': [[24, 34, 'MEASUREMENT'], [103, 111, 'MEASUREMENT'], [140, 147, 'MEASUREMENT']]}
{'entities': [[48, 52, 'CARDINAL'], [76, 96, 'DIMENSION']]}
{'entities': [[31, 44, 'DIMENSION']]}
{'entities': [[24, 27, 'CARDINAL'], [41, 44, 'CARDINAL']]}
{'entities': [[19, 22, 'CARDINAL']]}
{'entities': [[40, 46, 'CARDINAL'], [48, 58, 'MEASUREMENT'], [144, 160, 'DIMENSION']]}
{'entities': [[27, 32, 'CARDINAL'], [156, 157, 'CARDINAL']]}
{'entities': [[8, 9, 'CARDINAL']]}
{'entities': [[36, 46, 'MEASUREMENT']]}
{'entities': [[68, 74, 'MEASUREMENT']]}
{'entities': [[14, 20, 'MEASUREMENT']]}
{'e